In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import time
import src.gen_source as gen_source

import tcp_async
import tcp_sync

from live_analyse_py import live_analyse_py

In [ ]:
try:
    __IPYTHON__
    reload(tcp_async)
    reload(tcp_sync)
    reload(gen_source)
except:
    pass

In [ ]:
sync = tcp_sync.UhdSyncMsg(packet_size=4*8192,
                           packet_type="".join(["f"]*8192))
async = tcp_async.UhdAsyncMsg()

In [ ]:
top = live_analyse_py()

In [ ]:
samp_rate = 4000000 #samples / second
frequency_0  = samp_rate/10. #cycles / second
frequency_1  = samp_rate/9. #cycles / second
fft_size = 8192

In [ ]:
top.start()

In [ ]:
sync.has_msg()

In [ ]:
import peakutils
def score(msg, frequency_0, frequency_1, samp_rate):
    frequency_0_idx = np.round(frequency_0 / samp_rate * fft_size).astype(int)
    frequency_1_idx = np.round(frequency_1 / samp_rate * fft_size).astype(int)
    frequency_idx_diff = frequency_1_idx - frequency_0_idx
    
    start = frequency_0_idx % frequency_idx_diff + frequency_idx_diff / 2.
    start = np.round(start).astype(int)
    end   = msg.shape[0] - frequency_idx_diff - (msg.shape[0] - start) % frequency_idx_diff
    
    msg_cut = msg[start:end]
    
    msg_cut = msg_cut.reshape(-1, frequency_idx_diff)
    
    msg_peaks = np.max(msg_cut, axis=1)
    #plt.plot(msg_peaks, "ro")
    
    inter_mod = \
    (0.5 * (msg_peaks[8]/msg_peaks[7] + msg_peaks[9]/msg_peaks[10]),
     0.5 * (msg_peaks[8]/msg_peaks[6] + msg_peaks[9]/msg_peaks[11]),
     0.5 * (msg_peaks[8]/msg_peaks[5] + msg_peaks[9]/msg_peaks[12]),
     0.5 * (msg_peaks[8]/msg_peaks[4] + msg_peaks[9]/msg_peaks[13]),
     0.5 * (msg_peaks[8]/msg_peaks[3] + msg_peaks[9]/msg_peaks[14]),
     0.5 * (msg_peaks[8]/msg_peaks[2] + msg_peaks[9]/msg_peaks[15]),
     0.5 * (msg_peaks[8]/msg_peaks[1] + msg_peaks[9]/msg_peaks[16]))
    
    return(inter_mod)


In [ ]:
top.set_txgain(85)
for x2 in np.linspace(-0.15, 0.15, num = 15):
    t2 = time.time()
    for x3 in np.linspace(-0.15, 0.15, num = 14):
        for x4 in np.linspace(-0.15, 0.15, num = 7):
            path = gen_source.gen_file(frequency_0=frequency_0,
                                       frequency_1=frequency_1,
                                       samp_rate=samp_rate,
                                       x2 = x2,
                                       x3 = x3,
                                       x4 = x4,
                                       path='./np_twotone')
            #a_load = np.fromfile(path, dtype=np.complex64)
            #plt.plot(a_load.real[0:100])
            
            sync.has_msg()
            np.array(sync.get_msgs(2))
            msgs = np.array(sync.get_msgs(5))
            inter_mod_avg = (np.max(score(msgs[0], frequency_0, frequency_1, samp_rate)) 
                       + np.max(score(msgs[1], frequency_0, frequency_1, samp_rate)) 
                       + np.max(score(msgs[2], frequency_0, frequency_1, samp_rate)) 
                       + np.max(score(msgs[3], frequency_0, frequency_1, samp_rate)) 
                       + np.max(score(msgs[4], frequency_0, frequency_1, samp_rate)))/5
            msg = msgs[0]
            
            
            print("inter_mod %.3f, x_2 %.3f, x_3 %.3f" % (inter_mod_avg, x2, x3))
            
            plt.plot(msg)
            plt.savefig("/tmp/out/inter_mod_%.3f_x_2_%.3f_x_3_%.3f_x_4_%.3f.png" % (inter_mod_avg, x2, x3, x4))
            plt.clf()
        print(time.time() - t2)

In [ ]:
sync.stop()
async.stop()
top.stop()
top.wait()

In [ ]:
path = gen_source.gen_file(frequency_0=frequency_0,
                           frequency_1=frequency_1,
                           x1 = 0,
                           x2 = 0,
                           x3 = 0,
                           x4 = 0,
                           samp_rate=samp_rate,
                           path='./np_twotone')
a_load = np.fromfile(path, dtype=np.complex64)
plt.plot(a_load.real[0:100])